In [44]:
import pandas as pd
import pickle
from sklearn import metrics

In [45]:
# cargamos el mejor modelo
with open('datos/robust.pkl', 'rb') as robust:
    robust = pickle.load(robust)

In [46]:
with open("datos/['cut']scaler.pkl", 'rb') as cut_scaler:
    cut_scaler = pickle.load(cut_scaler)

In [47]:
with open("datos/['color']scaler.pkl", 'rb') as color_scaler:
    color_scaler = pickle.load(color_scaler)

In [48]:
with open('datos/decissiontree1.pkl', 'rb') as decissiontree1:
    decissiontree1 = pickle.load(decissiontree1)

In [49]:
df_test=pd.read_csv("datos/test.csv")
df_test.head()

,id,carat,cut,color,clarity,depth,table,x,y,z
0,0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.70
1,1,1.24,Premium,I,SI1,62.9,60.0,6.80,6.74,4.26
2,2,1.66,Premium,D,SI1,62.0,59.0,7.55,7.60,4.70
3,3,0.75,Premium,D,SI2,60.6,56.0,5.94,5.90,3.59
4,4,1.50,Fair,E,SI2,64.8,55.0,7.26,7.15,4.67


In [50]:
df_test.drop("id", axis = 1, inplace = True)
df_test.head(1)

,carat,cut,color,clarity,depth,table,x,y,z
0,0.32,Ideal,I,SI1,60.5,58.0,4.43,4.49,2.7


In [51]:
clarity_dict = {"I1": 2, "SI2" :2, "SI1" :1, "VS2" :1, "VS1" :1, "VVS2" :0, "VVS1": 0, "IF": 0}

df_test['clarity_ord'] = df_test['clarity'].replace(clarity_dict)
# Eliminar las variables originales
df_test.drop(['clarity'], axis=1, inplace=True)

In [52]:
df_test.head()

,carat,cut,color,depth,table,x,y,z,clarity_ord
0,0.32,Ideal,I,60.5,58.0,4.43,4.49,2.70,1
1,1.24,Premium,I,62.9,60.0,6.80,6.74,4.26,1
2,1.66,Premium,D,62.0,59.0,7.55,7.60,4.70,1
3,0.75,Premium,D,60.6,56.0,5.94,5.90,3.59,2
4,1.50,Fair,E,64.8,55.0,7.26,7.15,4.67,2


In [53]:
# estandarizamos todas las columnas que son de tipo numérico usando el transformer que nos guardamos cuando hicimos el modelo

estandarizadas = pd.DataFrame(robust.transform(df_test[["carat","depth","table","x","y","z"]]), columns = ["carat","depth","table","x","y","z"])
estandarizadas.head(1)

,carat,depth,table,x,y,z
0,-0.59375,-0.866667,0.333333,-0.688525,-0.664835,-0.732143


In [54]:
# estandarizamos todas las columnas que son de tipo numérico usando el transformer que nos guardamos cuando hicimos el modelo

cut = pd.DataFrame(cut_scaler.transform(df_test[["cut"]]).toarray(), columns=['cut_Fair', 'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good'])
cut.head(1)

,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good
0,0.0,0.0,1.0,0.0,0.0


In [55]:
color = pd.DataFrame(color_scaler.transform(df_test[["color"]]).toarray(), columns=['color_D', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I',
       'color_J'])
color.head(1)

,color_D,color_E,color_F,color_G,color_H,color_I,color_J
0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [56]:
# unimos todos los conjuntos de datos que hemos ido generando con los transformers
df_final  = pd.concat([estandarizadas, cut, color], axis = 1)
df_final.head(1)

,carat,depth,table,x,y,z,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J
0,-0.59375,-0.866667,0.333333,-0.688525,-0.664835,-0.732143,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [57]:
df_final["clarity_ord"]= df_test["clarity_ord"]
df_final.columns

Index(['carat', 'depth', 'table', 'x', 'y', 'z', 'cut_Fair', 'cut_Good',
       'cut_Ideal', 'cut_Premium', 'cut_Very Good', 'color_D', 'color_E',
       'color_F', 'color_G', 'color_H', 'color_I', 'color_J', 'clarity_ord'],
      dtype='object')

In [58]:
df_final=df_final.reindex(['carat', 'depth', 'table', 'x', 'y', 'z',  'clarity_ord',
       'cut_Fair', 'cut_Good', 'cut_Ideal', 'cut_Premium', 'cut_Very Good',
       'color_D', 'color_E', 'color_F', 'color_G', 'color_H', 'color_I',
       'color_J'],axis=1)

In [59]:
df_final.head(1)

,carat,depth,table,x,y,z,clarity_ord,cut_Fair,cut_Good,cut_Ideal,cut_Premium,cut_Very Good,color_D,color_E,color_F,color_G,color_H,color_I,color_J
0,-0.59375,-0.866667,0.333333,-0.688525,-0.664835,-0.732143,1,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0


In [61]:
df_prediccion1=pd.DataFrame(decissiontree1.predict(df_final))

In [63]:
df_prediccion1.head()

,0
0,6.098
1,8.608
2,9.668
3,7.789
4,8.898


In [67]:
df_prediccion1=df_prediccion1.reset_index()

In [68]:
df_prediccion1

,index,0
0,0,6.098
1,1,8.608
2,2,9.668
3,3,7.789
4,4,8.898
...,...,...
13480,13480,8.857
13481,13481,8.404
13482,13482,6.417
13483,13483,8.761


In [71]:
df_prediccion1.columns=["id","price"]
df_prediccion1.head()

,id,price
0,0,6.098
1,1,8.608
2,2,9.668
3,3,7.789
4,4,8.898


In [74]:
df_prediccion1.to_csv("datos/modelo1.csv", index=False)